## Module 1B
## Tutorial 1 covers how to organize your own images for use with CellOrganizer and illustrates building different types of models. Typically, telling CellOrganizer what files to use to construct a model is done using wildcards (e.g., “Expt1Sample*Nuclear.tif”), and it is helpful to organize your files to facilitate this.
## Module 1B builds a better model using SPHARM-RPDM for both the nucleus and the cell surface but to save compute time downsamples the images before building the model.
--> to make Module 1B from Module 1A, just uncomment options for SPHARM-RPDM postprocessing

--> change 'downsampling' to 16,16,4

--> change 'cell.type' from 'ratio' to 'spharm_rpdm' and

--> change 'nucleus_type' from 'cylindrical surface' to 'spharm_rpdm'

### Run the cell below to set up basic parameters:

In [25]:
from cellorganizer.tools import img2slml
import datetime
import glob
import urllib.request
import os
import uuid

model_name = "Module1B"
module_name = model_name
topdir = "/home/murphylab/cellorganizer/local/results"
outputdir = topdir + "/" + module_name
if not os.path.exists(topdir):
    os.makedirs(topdir)
os.chdir(topdir)
os.system("ls")

options = {}

#set general options
options["model.name"] = module_name
options["model.filename"] = module_name + '.mat'
options["output_filename"] = module_name 
options["verbose"] = 0
options["debug"] = 0
options["display"] = 0
options["model.id"] = str (uuid.uuid1())
options["downsampling"] = [16, 16, 4]
options["model.resolution"] = [0.049, 0.049, 0.2000]
options["if_skip_cell_nuclear_model"] = 0

# these options are only used for SPHARM-RPDM models
# postprocess of parameterization: alignment
options["model.spharm_rpdm.postprocess"] = 1
# # alignment method: 'major_axis' or 'foe'
options["model.spharm_rpdm.alignment_method"] = 'major_axis'
# # plane of rotation: 'xy', 'xz', 'yz' or 'xyz'
options["model.spharm_rpdm.rotation_plane"] = 'xy'
# # degree of the descriptor
options["model.spharm_rpdm.maxDeg"] = 31
# # latent dimension for the model
options["model.spharm_rpdm.latent_dim"] = 15
options["model.spharm_rpdm.segminnucfraction"] = 0.1

### Set up what kind of model is being created:

In [26]:
#set options that control what kind of model is created
options["train.flag"] = 'framework'
options["cell.class"] = 'cell_membrane'
#simplest cell shape model - ratio to nuclear model
#options["cell.type"] = 'ratio'
#best cell shape model - spharm_rpdm - see demo3D50 and demo3D52 for more options
options["cell.type"] = 'spharm_rpdm'
# if we are going to include nuclear/DNA images
options["nucleus.class"] = 'nuclear_membrane'
#simplest nuclear shape model - cylindrical surface
#options["nucleus.type"] = 'cylindrical_surface'
#best nuclear shape model - spharm_rpdm - see demo3D52 for more options
options["nucleus.type"] = 'spharm_rpdm'
#options["model.spharm_rpdm.components"] = {'cell'}
options["model.spharm_rpdm.components"] = {'nuc', 'cell'}
options["hd_threshold"] = 10
options["hd_thresh"] = 10

### Ideally, put all files to be used to build a model into a single directory (but all files for a single channel could be in a subdirectory).  The idea is to allow wildcards to be used easily.  Three examples are shown below for different ways that cells and channels might be organized into files.

### The examples use provided images - you can change the directory and filenames for your own data

### >>> You can change the method number below to choose a different method appropriate for your files <<<

In [27]:
options["masks"] = []
dnaImagesDirectoryPath = []
cellImagesDirectoryPath = []
method=1

### Single file for each cell and channel.  In this case, no masks are needed to identify single cells in multicell images.

In [28]:
if method==1:
    directory = '/home/murphylab/cellorganizer/local/images/HeLa/3D/processed/'

    file_pattern = 'LAM_cell1?_ch1_t1.tif'
    for name in glob.glob(directory + file_pattern):
        cellImagesDirectoryPath.append(name)

    file_pattern = 'LAM_cell1?_ch0_t1.tif'
    for name in glob.glob(directory + file_pattern):
        dnaImagesDirectoryPath.append(name)

    cellImagesDirectoryPath.sort()
    dnaImagesDirectoryPath.sort()

    cellImagesDirectoryPath = cellImagesDirectoryPath[0:3]
    dnaImagesDirectoryPath = dnaImagesDirectoryPath[0:3]

### Single file for each channel, but cropping is needed to ensure that only a single cell is processed from each image.   This case adds a file with a single binary mask for each cell.

In [29]:
if method==2:
    directory = '/home/murphylab/cellorganizer/local/images/HeLa/3D/processed/'

    file_pattern = 'LAM_cell1?_mask_t1.tif'
    for name in glob.glob(directory + file_pattern):
        options["masks"].append(name)

    file_pattern = 'LAM_cell1?_ch1_t1.tif'
    for name in glob.glob(directory + file_pattern):
        cellImagesDirectoryPath.append(name)

    file_pattern = 'LAM_cell1?_ch0_t1.tif'
    for name in glob.glob(directory + file_pattern):
        dnaImagesDirectoryPath.append(name)

    cellImagesDirectoryPath.sort()
    dnaImagesDirectoryPath.sort()
    options["masks"].sort()

    cellImagesDirectoryPath = cellImagesDirectoryPath[0:3]
    dnaImagesDirectoryPath = dnaImagesDirectoryPath[0:3]

### Multiple cells with multiple channels using OME-TIFF files with ROIs (Regions of Interests).  This is the most common case.  The channel number and time point to use from each file is specified using colons.  First number is channel, second number is timepoint.

In [30]:
if method==3:
    directory = '/home/murphylab/cellorganizer/local/images/ometiff_with_rois/'

    file_pattern = 'hela_3d_processed_lam_cell?.ome.tif'
    for name in glob.glob(directory + file_pattern):
        cellImagesDirectoryPath.append(name + ':1:0')
        dnaImagesDirectoryPath.append(name + ':2:0')

    cellImagesDirectoryPath.sort()
    dnaImagesDirectoryPath.sort()

In [31]:
outputdir = topdir + "/" + module_name + "method" + str(method)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
os.chdir(outputdir)
os.system("ls")
answer = img2slml('3D', dnaImagesDirectoryPath, cellImagesDirectoryPath, [], options)

CellOrganizer v2.10 (MATLAB RO2019a)


In [32]:
if answer:
    print('The model ' + model_name + '.mat has been created in ' + outputdir)

The model Module1B.mat has been created in /home/murphylab/cellorganizer/local/results/Module1Bmethod1
